In [32]:
import json
import pandas as pd

# Ler o JSON
with open('./pokedex-db.json', "r", encoding="utf-8") as f:
    pokedex_data = json.load(f)

# Converter para DataFrame
df = pd.DataFrame(pokedex_data)

# Exibir tamanho do DataFrame
print(f"Tamanho do DataFrame: {df.shape}")


Tamanho do DataFrame: (1025, 14)


In [ ]:
import requests
import time

# Adicionar coluna para localizações de captura
df['capture_location'] = None

# Resgatando Localizações de Captura 
def get_capture_locations(number):
    url = f"https://pokeapi.co/api/v2/pokemon/{number}/encounters"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            print(f"Requisição bem-sucedida para o Pokémon número {number}.")
        
            # Extrai o nome dos locais de captura
            locations = [loc['location_area']['name'] for loc in data]
            return locations
        else:
            return []
    except Exception as e:
        return []
    
# Para cada item da base de dados, resgata as localizações de captura
for index, row in df.iterrows():
    number = row['id']   
    print(f"Resgatando localizações de captura para o Pokémon número {number}/{len(df.index)}...")
    locations = get_capture_locations(number)
    print(f"Localizações de captura para o Pokémon número {number}: {locations}")
    df.at[index, 'capture_location'] = locations
    # time.sleep(1)  # Respeitar o limite de requisições
    
# Salvar o DataFrame atualizado em um novo arquivo JSON
df.to_json('./pokedex-db-with-capture-locations.json', orient='records', indent=4, force_ascii=False)
 
print("Arquivo JSON atualizado salvo como 'pokedex-db-with-capture-locations.json'.")
print(f"Tamanho do DataFrame atualizado: {df.shape}")

In [ ]:
# One-hot encoding para cada campo de array

# Garante que os campos de array estejam como listas (caso estejam ausentes)
for col in ['abilities', 'type', 'weakness', 'capture_location']:
    df[col] = df[col].apply(lambda x: x if isinstance(x, list) else [])


# Extrair valores únicos de habilidades, tipos, fraquezas e localizações de captura
abilities_dummies = df['abilities'].explode().str.strip().str.lower().str.title().dropna().unique()
type_dummies = df['type'].explode().str.strip().str.lower().str.title().dropna().unique()
weakness_dummies = df['weakness'].explode().str.strip().str.lower().str.title().dropna().unique()
capture_location_dummies = df['capture_location'].explode().str.strip().str.lower().str.title().dropna().unique()

# Para cada habilidade, tipo, fraqueza e localização de captura, cria colunas one-hot verificando se o valor está presente
df_abilities = df['abilities'].apply(lambda x: pd.Series({f'ability_{a}': int(a in x) for a in abilities_dummies}))
df_type = df['type'].apply(lambda x: pd.Series({f'type_{t}': int(t in x) for t in type_dummies}))
df_weakness = df['weakness'].apply(lambda x: pd.Series({f'weakness_{w}': int(w in x) for w in weakness_dummies}))
df_capture_location = df['capture_location'].apply(lambda x: pd.Series({f'location_{loc}': int(loc in x) for loc in capture_location_dummies}))

# Concatenar tudo
df_final = pd.concat([
    df.drop(['abilities', 'type', 'weakness', 'capture_location'], axis=1),
    df_abilities,
    df_type,
    df_weakness,
    df_capture_location
], axis=1)

# Salvar em CSV
df_final.to_csv('pokedex-db-parsed.csv', index=False, encoding='utf-8')

print("CSV com one-hot encoding gerado com sucesso!")
print(f"Tamanho do DataFrame final: {df_final.shape}\n\n")
print(df_final[['id', 'name'] + list(df_abilities.columns) + list(df_type.columns) +
                list(df_weakness.columns) + list(df_capture_location.columns)])

CSV com one-hot encoding gerado com sucesso!
Tamanho do DataFrame final: (1025, 1188)


        id          name  ability_Overgrow  ability_Blaze  ability_Torrent  \
0        1     Bulbasaur                 1              0                0   
1        2       Ivysaur                 1              0                0   
2        3      Venusaur                 1              0                0   
3        4    Charmander                 0              1                0   
4        5    Charmeleon                 0              1                0   
...    ...           ...               ...            ...              ...   
1020  1021   Raging Bolt                 0              0                0   
1021  1022  Iron Boulder                 0              0                0   
1022  1023    Iron Crown                 0              0                0   
1023  1024     Terapagos                 0              0                0   
1024  1025     Pecharunt                 0            